notes: 
    
* source activate cpac
* conda install jupyter
* git clone https:nbpapaya
* remember to do submodules


In [12]:
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [13]:
import urllib

pheno_file_url="https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b_preprocessed1.csv"
pheno_file_local="Phenotypic_V1_0b_preprocessed1.csv"

urllib.urlretrieve(pheno_file_url, pheno_file_local)

('Phenotypic_V1_0b_preprocessed1.csv',
 <httplib.HTTPMessage instance at 0x7f167c4356c8>)

In [14]:
import pandas as pd
pheno_df=pd.read_csv(pheno_file_local)
pheno_df.head()

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [15]:
pipes=['ccs','cpac','dparsf','niak']
strats=['filt_global','filt_noglobal','nofilt_global','nofilt_noglobal']
derivatives=['alff']#,'degree_binarize','degree_weighted','eigenvector_binarize',
    #'eigenvector_weighted','falff','func_mean','lfcd','reho','vmhc']
file_id = ["Pitt_0050003"]

In [16]:
import itertools

In [17]:
new_df = pd.DataFrame(columns = ["pipe", "strat", "deriv", "file_id"])
for combo in itertools.product(pipes, strats, derivatives, file_id):
    new_df = new_df.append(pd.Series({key: combo[i] for i, key in enumerate(new_df.columns)}), ignore_index=True)

In [18]:
new_df.head()

,pipe,strat,deriv,file_id
0,ccs,filt_global,alff,Pitt_0050003
1,ccs,filt_noglobal,alff,Pitt_0050003
2,ccs,nofilt_global,alff,Pitt_0050003
3,ccs,nofilt_noglobal,alff,Pitt_0050003
4,cpac,filt_global,alff,Pitt_0050003


In [21]:
# configure out download string
template_str="https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative\
/Outputs/{pipeline}/{strategy}/{derivative}/{file_id}_{derivative}.nii.gz"

def get_data_str(pipe, strat, deriv, file_id):
    data_str = template_str.format(pipeline=pipe, strategy=strat, derivative=deriv, file_id=file_id)
    local_file = os.path.join("data",os.path.basename(data_str))
    if not os.path.isfile(local_file):
        # now download the file
        try:
            urllib.urlretrieve(data_str, local_file)
        except Exception as e:
            print "Could not download %s"%(data_str)
            pheno_df.loc[pheno_df["FILE_ID"]==file_id,"dl_error"]=True
            
    return data_str, local_file

In [22]:
new_df["local_filename"] = new_df.index.map(lambda i: get_data_str(new_df.loc[i].pipe,
                                                             new_df.loc[i].strat, 
                                                             new_df.loc[i].deriv, 
                                                             new_df.loc[i].file_id)[1])
new_df["remote_filename"] = new_df.index.map(lambda i: get_data_str(new_df.loc[i].pipe,
                                                             new_df.loc[i].strat, 
                                                             new_df.loc[i].deriv, 
                                                             new_df.loc[i].file_id)[0])

In [40]:
new_df.head()

,pipe,strat,deriv,file_id,filename,local_filename,remote_filename
0,ccs,filt_global,alff,Pitt_0050003,data/Pitt_0050003_alff.nii.gz,data/Pitt_0050003_alff.nii.gz,https://s3.amazonaws.com/fcp-indi/data/Project...
1,ccs,filt_noglobal,alff,Pitt_0050003,data/Pitt_0050003_alff.nii.gz,data/Pitt_0050003_alff.nii.gz,https://s3.amazonaws.com/fcp-indi/data/Project...
2,ccs,nofilt_global,alff,Pitt_0050003,data/Pitt_0050003_alff.nii.gz,data/Pitt_0050003_alff.nii.gz,https://s3.amazonaws.com/fcp-indi/data/Project...
3,ccs,nofilt_noglobal,alff,Pitt_0050003,data/Pitt_0050003_alff.nii.gz,data/Pitt_0050003_alff.nii.gz,https://s3.amazonaws.com/fcp-indi/data/Project...
4,cpac,filt_global,alff,Pitt_0050003,data/Pitt_0050003_alff.nii.gz,data/Pitt_0050003_alff.nii.gz,https://s3.amazonaws.com/fcp-indi/data/Project...


In [1]:
from nbpapaya import Brain

In [7]:
Brain([os.path.abspath("./data/Pitt_0050003_alff.nii.gz")], 
      host="https://ec2-52-90-103-243.compute-1.amazonaws.com")

doing checks /root/.jupyter/custom/
